In [19]:
import tensorflow as tf
import time
tf.__version__

'1.12.0'

### 1. Hello World!

상수 h, w를 정의하고 결합한다. Session 객체는 외부의 텐서플로 연산 메커니즘에 대한 인터페이스 역할을 하며, 이를 이용해 우리가 정의한 연산 그래프를 실행한다.

In [2]:
h = tf.constant("Hello")
w = tf.constant(" World!")
hw = h + w

with tf.Session() as sess:
    ans = sess.run(hw)

print(ans)

b'Hello World!'


In [4]:
ph = "Hello"
pw = " World!"
phw = ph+pw
print(phw)
print(hw)

Hello World!
Tensor("add:0", shape=(), dtype=string)


### MNIST

필기체 숫자로 이로어진 MNIST DB는 이미지 처리와 머신러닝 예제로 많이 사용되어 왔다. 이 예제로 시험삼아 돌려보자

#### 소프트맥스 회귀
소프트맥스 회귀모형은 이미지의 위치에 따른 각 필셀에 해당하는 값이 각 숫자에 따라 높은지 낮은지 경향성을 계산한다. 예를 들어 i, j 픽셀이 칠해져 있을 경우 0일 확률 n%, ... , 9일 확률 n%를 따진다. 이렇게 모든 픽셀을 따져 숫자를 결정한다.

In [18]:
# For MNIST DATA Download
from tensorflow.examples.tutorials.mnist import input_data


DATA_DIR = '/tmp/data'
NUM_STEPS = 500000 #이동 단계 횟수
MINIBATCH_SIZE = 100 # 각 단계에서 사용할 예제의 수

# MNIST DATA Download
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


- Variable: 연산 과정에서 조작되는 값

- placeholder: 연산 그래프가 실행될 떄 제공되어야 하는 값

이미지 자체(x)는 연산 그래프가 실행될 때 제공되야하므로 placeholder이다. 

크기를 나타내는 [None, 784]가 의미하는 것은 한 이미지의 사이즈가 784(하나의 벡터에 28*28 픽셀)이며 얼마나 많은 이미지를 한 번에 사용할지는 이 시점에서 정하지 않겠다(None)라는 뜻

In [7]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))

MNIST는 지도학습이다. 그러므로 정답 레이블과 예측 레이블이 일치하도록 모델을 학습시켜야 한다.
y_true에서도 얼마나 많은 이미지를 한 번에 사용할지 모르므로 None, 정답 레이블은 0~9이므로 10, 정답의 유형은 float32

가설에 의해 계산된 값 y를 넣지 않고 tf.matmul(x, W) + b 를 넣은 이유는 아래에 나올tf.nn.softmax_cross_entropy_with_logits 함수가 softmax를 포함하기 때문이다. 

- y_true: 정답 레이블
- y_pred: 예측 레이블
- matmul: Matrix Multiplication. 행렬의 곱셈 역할

In [9]:
y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)

이 모델에서 교차 엔트로피라 불리는 유사성 척도를 사용한다. 이 모델의 출력 값이 각 분류에 대응되는 확률일 때 일반적으로 사용된다. 이 요소는 손실함수(loss functuion)이라 부르기도 한다.

- 손실함수: 함수의 결과를 실제 결과의 차를 측정하는 척도이다. 실제 널리 쓰이는 유사성 척도는 '차이의 절댓값'이지만 사용하기 여의치 않은 경우가 많아 이번엔 교차 엔트로피 같은 유사성 척도를 사용하였다.
- reduce_mean: 행렬의 평균 계산
- logits: 주어진 X에 대해서 학습할 W를 매트릭스로 곱하고 bias를 더하는 것

원래는 logits를 만들고 softmax로 통과시켜 hypothesis를 세워야한다. 하지만 친절한 softmax_cross_entropy_with_logits이 모든 것을 해주므로 logits의 인자로 y_pred만 넘기면 된다.





In [10]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))

학습 방법(손실 함수의 값을 최소화 하는 방법)에 관한것이다. 경사하강법을 사용하였으며 rate값을 0.2로 설정하였다.

In [15]:
gd_step = tf.train.GradientDescentOptimizer(0.2).minimize(cross_entropy)

모델의 정확도를 테스트하기 위해 사용할 평가 과정을 정의하였다. 정확하게 분류된 테스트 데이터의 비율을 사용하였다.

In [13]:
correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [20]:
start_time = time.time() 
with tf.Session() as sess:

    # Train
    sess.run(tf.global_variables_initializer()) # 모든 변수 초기화
    
    """모델을 만들 때 정의한 placeholder를 이용하여 연산을 수행하고자 할 때 feed_dict를 사용"""
    for _ in range(NUM_STEPS):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})

    """학습을 마친 모델을 평가하기 위해 앞서 정의한 정확도 계산 과정을 수행한다."""
    # Test
    ans = sess.run(accuracy, feed_dict={x: data.test.images, y_true: data.test.labels})

print("Accuracy: {:.4}%".format(ans*100))
print("--- %s seconds ---" %(time.time() - start_time))

Accuracy: 92.39%
--- 398.0545644760132 seconds ---
